In [1]:
import os
os.environ["OPENAI_API_KEY"]=""

# Setting environment parameter

In [2]:
# !pip install langchain_openai


In [3]:
# !pip install chromadb

In [4]:
# !pip install -U langchain-community

In [20]:
import warnings
warnings.filterwarnings('ignore')
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

CHROMA_PATH = ".\chroma6"


In [21]:
# !pip install pypdf

# load data into Chroma DB

In [22]:
# embeddings = OpenAIEmbeddings()
loaders = PyPDFLoader("/content/Panchatantra@RAG.pdf")

documents = loaders.load()

db = Chroma.from_documents(
    documents,
    OpenAIEmbeddings(),
    persist_directory=CHROMA_PATH
  )
db.persist()


In [23]:
import sys
sys.exit(1)7

SystemExit: 1

In [26]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [27]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [28]:
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  # YOU MUST - Use same embedding function as before
  embedding_function = OpenAIEmbeddings()

  # Prepare the database
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=3)

  # Check if there are any matching results or if the relevance score is too low
  # if len(results) == 0 or results[0][1] < 0.7:
  #   print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)

  # Initialize OpenAI chat model
  model = ChatOpenAI()

  # Generate response text based on the prompt
  response_text = model.predict(prompt)

   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]

  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  return formatted_response, response_text


# Ask Question

In [29]:
query_text="what are first 5 contents from PDF?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

1. The story of the monkey and the log
2. The story of the jackal and the drum
3. The story of the merchant Dantila
4. The story of the jackal and the Sanyasi
5. The story of the cobra and the crow


In [30]:
query_text="THE STORY OF THE JACKAL AND THE SANYASI what is the moral from this?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The moral of the story is not to trust everyone blindly, as people with ill intentions may take advantage of your trust and deceive you. It also teaches the importance of being vigilant and not being swayed by honeyed words or flattery.


In [31]:
query_text="whats placed in pgno 28"
formatted_response, response_text = query_rag(query_text)
print(response_text)

Based on the context provided, it is not mentioned what is placed on page 28.


In [32]:
query_text="FHE STORY OF THl!. COBRA AND THE CROW what are the characters placed there?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The characters are a pair of crows (husband and wife), a black cobra, and a jackal.


In [33]:
query_text="the book which i have provided is translated by? "
formatted_response, response_text = query_rag(query_text)
print(response_text)

The book provided in the context is translated by the author's "t•ru", Professor (Dr.) S.B. Hudlikar.


In [34]:
query_text="Provided book published by?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The book was published by Rupa Publications India Pvt. Ltd.


# 1. get the embeddings for the query
# 2. get the context from the chroma related to embeddings
# 3. formulate the context based on query
# 4. Ask the llm with the  context and query

In [36]:
query_text="provide me morals of stories?"
formatted_response, response_text = query_rag(query_text)
print(response_text)

The morals of the story could be:
1. Sometimes unexpected events or people can be beneficial in unexpected ways.
2. True intentions and loyalty are revealed during challenging situations.
3. It is important to treat others with kindness and respect, as it may lead to positive outcomes.
4. Seeking protection or help from unexpected sources can sometimes lead to positive results.
5. Treat others well, as you never know when you may need their help in the future.


In [1]:
!pip install faiss-gpu


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
